In [ ]:
import os
from pathlib import Path

# 获取当前工作目录
notebook_dir = Path(os.getcwd())

print(f"Notebook directory: {notebook_dir}")


Notebook directory: /home/leinao/code/Dspike2


In [2]:
import torch
import torch.nn as nn
from dataclasses import dataclass
from pathlib import Path



@dataclass
class MyModelConfig:
    input_size:int = 12
    hidden_size:int = 12
    output_size:int = 12

class MyModel(nn.Module):
    def __init__(self, config:MyModelConfig):
        super(MyModel, self).__init__()
        self.config = config
        self.fc1 = nn.Linear(config.input_size, config.hidden_size)
        self.fc2 = nn.Linear(config.hidden_size, config.output_size)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [3]:
import random
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from tqdm import tqdm
from darkit.core import Trainer, TrainerConfig

from typing import Optional

model_cache_path = notebook_dir / '.cache'

if not model_cache_path.exists():
    model_cache_path.mkdir()

print(f"model_cache_path: {model_cache_path}")

@dataclass
class MyTrainerConfig(TrainerConfig):
    device = "cuda"
    lr = 1e-3
    batch_size = 4
    max_step: int = 10000 # 定义最大训练步数
    save_step_interval: int = 10000 # 定义保存模型间隔


class MyTrainer(Trainer):
    def __init__(self, model:MyModel, config:MyTrainerConfig, **kwargs):
        super(MyTrainer, self).__init__(model, config, **kwargs)
        self.config = config
    
    @property
    def save_directory(self) -> Optional[Path]:
        model_name = self.config.name
        if model_name is not None and self._is_master_process():
            save_directory = model_cache_path / model_name
            return save_directory
        else:
            return None

    def _get_optimizer(self):
        return torch.optim.Adam(self.model.parameters(), lr=self.config.lr)
    
    def train(self, train_dataset, val_dataset=None):
        # 在这里实现自定义的训练逻辑
        dataloader = DataLoader(train_dataset, batch_size=self.config.batch_size)
        
        optimizer = self._get_optimizer()
        for step, batch in tqdm(enumerate(dataloader)):
            inputs, labels = batch
            outputs = self.model(inputs)
            loss = F.mse_loss(outputs, labels)
            loss.backward()
            optimizer.step()
            
            # 调用 Trainer 父类提供的保存和评估方法
            # 会根据 config 提供的参数来控制保存和评估的逻辑
            self.current_step = step
            self._auto_save_pretrained()

Trainer.register(MyModel.__name__, MyTrainer)

name = f"TEST_MODEL_{random.randint(10,99)}"
mconf = MyModelConfig()
model = MyModel(mconf)
tconf = MyTrainerConfig(name=name)
trainer = MyTrainer(model, tconf)

# 生成一些随机数据
x = torch.randn(12)
y = torch.randn(12)
train_dataset = [(torch.tensor(x), torch.tensor(y)) for _ in range(4*10000)]

trainer.train(train_dataset)

model_cache_path: /home/leinao/code/Dspike2/.cache
{'name': 'TEST_MODEL_71', 'max_step': 10000, 'save_step_interval': 10000}
Save failed: source code not available
Logger initialized at /home/leinao/code/Dspike2/.cache/TEST_MODEL_71/train_log.csv


/tmp/ipykernel_3539563/3213859862.py:71: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  train_dataset = [(torch.tensor(x), torch.tensor(y)) for _ in range(4*10000)]
10000it [00:02, 3811.25it/s]

Model saved at /home/leinao/code/Dspike2/.cache/TEST_MODEL_71
Model saved epoch 10000/10000 at iter-10000-ckpt


In [4]:
from darkit.core import Predicter

class MyPredicter(Predicter):
    def __init__(self, name, model, device="cpu"):
        super(MyPredicter, self).__init__(name, model, device)
        self.model = self.model.to(self.device)
    
    @classmethod
    def get_save_directory(cls, model_name) -> Path:
        save_directory = model_cache_path / model_name
        return save_directory
    
    @classmethod
    def get_model(cls, name:str, checkpoint: Optional[str] = None):
        # 在这里实现自定义的获取模型的逻辑
        checkpoint_path = cls.get_checkpoint(name, checkpoint)
        config_dict = cls.get_model_config_json(name)
        config = MyModelConfig(**config_dict)
        model = MyModel(config=config)
        checkpoint_dict = torch.load(checkpoint_path, weights_only=True)
        model.load_state_dict(checkpoint_dict["model"], strict=True)
        return model
    
    def predict(self, input):
        input = input.to(self.device)
        output = self.model(input)
        return output
    
Predicter.register(MyModel.__name__, MyPredicter)

predicter = MyPredicter.from_pretrained(name)
predicter.predict(torch.randn(12))

save_directory /home/leinao/code/Dspike2/.cache/TEST_MODEL_71
save_directory /home/leinao/code/Dspike2/.cache/TEST_MODEL_71


tensor([-1.0915,  0.3169,  0.0660,  0.2210,  0.5283,  0.2934, -0.2829,  0.4491,
         0.5927, -1.0472,  1.7596,  0.1983], device='cuda:0',
       grad_fn=<ViewBackward0>)